# Baseline: Reflex Control

Steers the car using a PD controller on the estimated $y_m$ track median offset.

In [3]:
from DriveItGym import *
import numpy as np
from numpy import pi

num_episodes = 1  # number of episodes to run
time_limit = 10.0
throttle_limit = 1.0
gamma = 0.98
epsilon = 0.03
kp, kv = 1.2, 0.2
random_position=False

env = DriveItEnv(time_limit, throttle_limit, gamma, trail_length=3.0)
observation = env._reset(random_position)
ym_ = 0.0
done = False
reward_episode = 0
reward_sum = 0.0

for i_episode in range(1, num_episodes + 1):
    while not done:
        if num_episodes < 2:
            env.render()

        xm, ym, theta, steer, throttle = observation
        dym = ym - ym_
        ym_ = ym
        f = steer + kp * ym + kv * dym / dt
        if f > epsilon: action = 2
        elif f < -epsilon: action = 1
        else: action = 0
            
        if throttle < throttle_limit:
            action += 3

        observation, reward, done, info = env.step(action)
        reward_episode += reward
        #reward_sum = reward + gamma * reward_sum

        #if info['checkpoint']: 
        #    print('checkpoint: reward: %f' % (reward_sum))
        #if info['lap']: 
        #    print('lap:        reward: %f' % (reward_sum))

    if num_episodes <= 10:
        print('Episode %d %s. reward: %f, laps: %f' % \
              (i_episode, info['done'], reward_episode, \
               reward_episode / lap_median_length))
    reward_sum += reward_episode
    reward_episode = 0
    observation = env._reset(random_position)
    done = False

print('Average reward: %f' %(reward_sum / num_episodes))

Episode 1 complete. reward: 17.828333, laps: 3.541379
Average reward: 17.828333


In [8]:
env.close()

In [6]:
env._reset(random_position)

array([ 0.,  0.,  0.,  0.,  0.])

In [7]:
env.render()

In [22]:
# out penalty = max discounted reward for median laps at full speed
rmax = throttle_limit * v_max * dt / (1 - gamma)
rmax

2.0833333333333313